In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime as dt

In [ ]:
# Our reference time period
START = dt(1999, 1, 1)
END = dt(2017, 12, 31)

In [ ]:
# we create a time range (timeindex) that we will use in a while
ref_period = pd.date_range(START, END)

In [ ]:
## RETRIEVE DATA FOR NASDAQ STOCKS FROM QUANDL (might take a while)

In [ ]:
# Microsoft Corp. stock (listed since 1986) 
msft = pdr.DataReader('MSFT', 'quandl', START, END)

In [ ]:
# Amazon.com Inc. stock (listed since 2000)
amzn = pdr.DataReader('AMZN', 'quandl', START, END)

In [ ]:
# Alphabet Inc. (Google's holding) stock (listed since 2004)
googl = pdr.DataReader('GOOGL', 'quandl', START, END)

In [ ]:
# Facebook Inc. stock (listed since 2012)
fb = pdr.DataReader('FB', 'quandl', START, END)

In [ ]:
## DROP UNUSED COLUMNS: "ExDividend" AND "SplitRatio" 

In [ ]:
del msft['ExDividend']
del amzn['ExDividend']
del googl['ExDividend']
del fb['ExDividend']

In [ ]:
del msft['SplitRatio']
del amzn['SplitRatio']
del googl['SplitRatio']
del fb['SplitRatio']

In [ ]:
## REBASE DATAFRAMES ON REFERENCE PERIOD

# this will add/remove rows based on the actual dataframe and 
# will introduce NaN's on days for which data is not available

In [ ]:
msft = msft.reindex(ref_period)
amzn = amzn.reindex(ref_period)
googl = googl.reindex(ref_period)
fb = fb.reindex(ref_period)

In [ ]:
## CALCULATE HIGH/LOW DAILY PEAKS FOR EACH STOCK
daily_highs = {
    'msft': msft['High'].max(),
    'amzn': amzn['High'].max(),
    'googl': googl['High'].max(),
    'fb': fb['High'].max()
}
daily_lows = {
    'msft': msft['Low'].min(),
    'amzn': amzn['Low'].min(),
    'googl': googl['Low'].min(),
    'fb': fb['Low'].min()  
}

In [ ]:
# then derive highest/lowest daily peaks in the set
highest_daily_peak = max(daily_highs.keys(), key=(lambda key: daily_highs[key]))
lowest_daily_peak = min(daily_lows.keys(), key=(lambda key: daily_lows[key]))
highest_daily_peak, lowest_daily_peak

In [ ]:
## CALCULATE WHETHER EACH STOCK CLOSED ABOVE/BELOW THE AVERAGE DAILY CLOSING PRICE
## ON EACH DAY

In [ ]:
# first, let's create a new dataframe with daily closing prices
daily_closes = pd.DataFrame({
    'msft': msft['Close'],
    'amzn': amzn['Close'],
    'fb': fb['Close'],
    'googl': googl['Close']
})

In [ ]:
# now calculate the average closing price for every day and store it
daily_closes['avg'] = daily_closes.mean(axis=1)

In [ ]:
# then check when the daily closing price is above the daily average and store it
daily_closes['msft_above_daily_avg'] = msft['Close'] > daily_closes['avg']
daily_closes['amzn_above_daily_avg'] = amzn['Close'] > daily_closes['avg']
daily_closes['googl_above_daily_avg'] = googl['Close'] > daily_closes['avg']
daily_closes['fb_above_daily_avg'] = fb['Close'] > daily_closes['avg']


In [ ]:
# ...and the winner is...
days_above_daily_avg = {
    'msft': daily_closes['msft_above_daily_avg'].sum(),
    'amzn': daily_closes['amzn_above_daily_avg'].sum(),
    'googl': daily_closes['googl_above_daily_avg'].sum(),
    'fb': daily_closes['fb_above_daily_avg'].sum()  
}
most_durable_above_daily_avg = max(days_above_daily_avg.keys(),
                                   key=(lambda key: days_above_daily_avg[key]))
most_durable_above_daily_avg

In [ ]:
## UNDERSAMPLE DATA: 1-MONTH WIDE BINS, FILLED WITH MEAN CLOSE PRICE ON THE PERIOD

In [ ]:
monthly_msft = msft.resample('M').mean()
monthly_amzn = amzn.resample('M').mean()
monthly_googl = googl.resample('M').mean()
monthly_fb = fb.resample('M').mean()

In [ ]:
## CALCULATE HIGH/LOW MONTHLY PEAKS FOR EACH STOCK
monthly_highs = {
    'msft': monthly_msft['High'].max(),
    'amzn': monthly_amzn['High'].max(),
    'googl': monthly_googl['High'].max(),
    'fb': monthly_fb['High'].max()
}
monthly_lows = {
    'msft': monthly_msft['Low'].min(),
    'amzn': monthly_amzn['Low'].min(),
    'googl': monthly_googl['Low'].min(),
    'fb': monthly_fb['Low'].min()  
}

In [ ]:
# then derive highest/lowest monthly peaks in the set
highest_monthly_peak = max(monthly_highs.keys(), key=(lambda key: monthly_highs[key]))
lowest_monthly_peak = min(monthly_lows.keys(), key=(lambda key: monthly_lows[key]))
highest_monthly_peak, lowest_monthly_peak

In [ ]:
## CALCULATE WHETHER EACH STOCK CLOSED ABOVE/BELOW THE AVERAGE MONTHLY CLOSING PRICE
## ON EACH MONTH

In [ ]:
# first, let's create a new dataframe with monthly closing prices
monthly_closes = pd.DataFrame({
    'msft': monthly_msft['Close'],
    'amzn': monthly_amzn['Close'],
    'fb': monthly_fb['Close'],
    'googl': monthly_googl['Close']
})

In [ ]:
# now calculate the average closing price for every month and store it
monthly_closes['avg'] = monthly_closes.mean(axis=1)

In [ ]:
# then check when the monthly closing price is above the monthly average and store it
monthly_closes['msft_above_monthly_avg'] = monthly_msft['Close'] > monthly_closes['avg']
monthly_closes['amzn_above_monthly_avg'] = monthly_amzn['Close'] > monthly_closes['avg']
monthly_closes['googl_above_monthly_avg'] = monthly_googl['Close'] > monthly_closes['avg']
monthly_closes['fb_above_monthly_avg'] = monthly_fb['Close'] > monthly_closes['avg']

In [ ]:
# ...and the winner is...
months_above_monthly_avg = {
    'msft': monthly_closes['msft_above_monthly_avg'].sum(),
    'amzn': monthly_closes['amzn_above_monthly_avg'].sum(),
    'googl': monthly_closes['googl_above_monthly_avg'].sum(),
    'fb': monthly_closes['fb_above_monthly_avg'].sum()  
}
most_durable_above_monthly_avg = max(months_above_monthly_avg.keys(),
                                   key=(lambda key: months_above_monthly_avg[key]))
most_durable_above_monthly_avg

In [ ]:
## SAVE ALL DATAFRAMES TO CSV FILES
msft.to_csv('msft.csv')
amzn.to_csv('amzn.csv')
googl.to_csv('googl.csv')
fb.to_csv('fb.csv')
monthly_msft.to_csv('monthly_msft.csv')
monthly_amzn.to_csv('monthly_amzn.csv')
monthly_googl.to_csv('monthly_googl.csv')
monthly_fb.to_csv('monthly_fb.csv')
daily_closes.to_csv('daily_closes.csv')
monthly_closes.to_csv('monthly_closes.csv')